# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察填補缺值以及 標準化 / 最小最大化 對數值的影響

# [作業重點]
- 觀察替換不同補缺方式, 對於特徵的影響 (In[4]~In[6], Out[4]~Out[6])
- 觀察替換不同特徵縮放方式, 對於特徵的影響 (In[7]~In[8], Out[7]~Out[8])

In [ ]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

In [2]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [3]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
df.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


# 作業1
* 試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?
* 答：<font color="red">我發現「空值補眾數」，效果最好</font>

In [9]:
# 作業1-1 : 空值補 -1, 做羅吉斯迴歸
df_m1 = df.fillna(-1)
train_X = df_m1[:train_num]
estimator = LogisticRegression(solver='lbfgs')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6982644788418415

In [13]:
# 作業1-2 : 空值補 0, 做羅吉斯迴歸
df_m1 = df.fillna(0)
train_X = df_m1[:train_num]
estimator = LogisticRegression(solver='lbfgs')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6993817972775958

In [14]:
# 作業1-3 : 空值補 1, 做羅吉斯迴歸
df_m1 = df.fillna(1)
train_X = df_m1[:train_num]
estimator = LogisticRegression(solver='lbfgs')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6993817972775958

In [15]:
# 補缺方式：補某個特定值（0、1、-1）、補平均、補中位數、補眾數

# 作業1-4 : 空值補平均
df_mean = df.fillna(df.mean())
train_X = df_mean[:train_num]
estimator = LogisticRegression(solver='lbfgs')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6959413955734954

In [16]:
# 補缺方式：補某個特定值（0、1、-1）、補平均、補中位數、補眾數

# 作業1-5 : 空值補中位數
df_median = df.fillna(df.median())
train_X = df_median[:train_num]
estimator = LogisticRegression(solver='lbfgs')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6970587140092496

In [17]:
# 補缺方式：補某個特定值（0、1、-1）、補平均、補中位數、補眾數

# 作業1-6 : 空值補眾數 (mean、media回傳Series，mode回傳的是dataframe)

df_mode = df.fillna(df.mode().iloc[0])

train_X = df_mode[:train_num]
estimator = LogisticRegression(solver='lbfgs')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7026769043193688

# 作業2
* 使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?
* 答：<font color="red">我發現「標準化縮放器」，效果最好（比使用原值略好一些）</font>

In [7]:
# 補值的部份，補眾數的效果最好，所以作業二裡，我以補眾數＋不同的標準化方式，進行比較
# 使用MinMax縮放器
import warnings
warnings.filterwarnings('ignore')

df_mode = df.fillna(df.mode().iloc[0])
df_temp = MinMaxScaler().fit_transform(df_mode)

train_X = df_temp[:train_num]
estimator = LogisticRegression(solver='lbfgs')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6971092852048754

In [8]:
# 補值的部份，補眾數的效果最好，所以作業二裡，我以補眾數＋不同的標準化方式，進行比較
# 使用標準化縮放器
import warnings
warnings.filterwarnings('ignore')

df_mode = df.fillna(df.mode().iloc[0])
df_temp = StandardScaler().fit_transform(df_mode)

train_X = df_temp[:train_num]
estimator = LogisticRegression(solver='lbfgs')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7026831813892325

### 答案填寫於問題下方的紅字區